In [1]:
# Merge 12 files
#### function name: merge_stu_log(files_path) , input: files path list, output: DataFrame of all data
import pandas as pd
import numpy as np

def merge_stu_log(files_path):
    # Initial DataFrame
    df = pd.DataFrame()
    log1 = pd.read_csv(files_path[0])
    initial_col_order = [col.lower() for col in list(log1.columns)]
    # in loop
    for file in files_path:
        #load csv file as pandas
        student_log = pd.read_csv(file)
        student_log.columns = [col.lower() for col in student_log]
        df = pd.concat([df, student_log])

    df = df[initial_col_order]
    return df
student_log_list = ["../dataset/anonymized_dataset_for_ADM2017/student_log_{}.csv".format(i) for i in range(1,13)]
student_log = merge_stu_log(student_log_list)

C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2850: DtypeWarning: Columns (74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


# student list and question list

In [2]:
# student list
student_list = student_log['itest_id'].unique()
print(len(student_list))

# question list 
question_list = student_log['problemid'].unique()
print(len(question_list))

1709
4117


# students information

In [3]:
stu_information_list = ['itest_id','aveknow','avecarelessness','avecorrect',
                        'numactions','averesbored','averesengcon',
                         'averesconf','averesfrust','averesofftask','averesgaming']
student_information = student_log.loc[:,stu_information_list].drop_duplicates()  
student_information.to_csv("../dataset/student_information.csv",index=False)

# question information

In [4]:
# here don't use problemType because of inconsistency, might need cleaning
que_information_list = ['problemid','skill']

que_information = student_log.loc[:,que_information_list].drop_duplicates()
que_information_copy = que_information.copy()

p = que_information.groupby(['problemid']).count()
p = p[p['skill']>1]
for pr in p.index:
    a = que_information[que_information['problemid']==pr]
    b = a[a['skill']=='noskill']
    que_information.drop(b.index,inplace=True)
    
reduced_set = set(question_list) - set(que_information['problemid'])
for redu in reduced_set:
    que_information = pd.concat([que_information,que_information_copy[que_information_copy['problemid']==redu]])

que_information.to_csv("../dataset/question_information.csv",index=False)    
    

# Matrixs by different features

In [91]:
import datetime
def endtime(timeStamp):
    dateArray = datetime.datetime.utcfromtimestamp(timeStamp)
    otherStyleTime = dateArray.strftime("%Y-%m-%d %H:%M:%S")
    return otherStyleTime
def f(x):
    return x[2]+x[3]+x[5]+x[6]+x[8]+x[9]
def f1(x):
    return x[2]+x[3]+x[5]+x[6]

# pick up columns that should be used to filter
columns_list = ['itest_id','actionid','problemid','timetaken','correct',
                'hint','bottomhint','frishelprequest','scaffold',
                'frWorkingInSchool'.lower(),'responseIsFillIn'.lower(),
                'frIsHelpRequestScaffolding'.lower(),'timeOver80'.lower(),'endtime']
students_actions = student_log.loc[:,columns_list]
students_actions['end_time']=students_actions['endtime'].apply(lambda x: endtime(x))
students_actions['end_time_day']=students_actions['end_time'].apply(f).astype(str).astype(int)
students_actions['end_time_month']=students_actions['end_time'].apply(f1).astype(str).astype(int)
group = students_actions.groupby(['itest_id','problemid'])
last = group.tail(1).set_index(['itest_id','problemid'])

### action counting

In [68]:
action_count = group.count().loc[:,['actionid']]
action_count = action_count.rename(columns={'actionid':'action_count'})

In [69]:
def find_time_to_hint_and_scaffold(datafr):

    if 1 in list(datafr['hint']):
        index = list(datafr['hint']).index(1)
    else:
        index = -1
    if 1 in list(datafr['scaffold']):
        if index > list(datafr['scaffold']).index(1):
            index = list(datafr['scaffold']).index(1)
    else:
        index = max(index,-1)
    if index == -1:
        return -1
    else:
        return sum(list(datafr['timetaken'][:index+1]))

### sum other features

In [93]:
sum_table = group.sum()
# the times of wrong action  = totalaction - action_request_hint - correct
sum_table['wrong'] = action_count['action_count'] - sum_table['hint'] - sum_table['correct']
sum_table['wrongrate'] = sum_table['wrong']/action_count['action_count']

# use hint rate
sum_table['hintrate'] = sum_table['hint']/action_count['action_count']

# correct could be more than once so the rate might more suitable
sum_table['correctrate'] = sum_table['correct']/action_count['action_count']

# first request help rate
sum_table['frhrate'] = sum_table['frishelprequest']/action_count['action_count']

# working in school rate (could be 1 or 0 in most of time)
sum_table['workinschoolrate'] = sum_table['frworkinginschool']/action_count['action_count']

# filedin (could be 1 or 0 in most of time)
sum_table['filedin'] = sum_table['responseisfillin']/action_count['action_count']

# first response is request scaffold help   (could be 1 or 0 in most of time)
sum_table['frscaffoldh'] = sum_table['frishelprequestscaffolding']/action_count['action_count']

# timeover80
sum_table['timeover80rate'] = sum_table['timeover80']/action_count['action_count']

# time to hint
sum_table['timetohint'] = group.apply(lambda x:find_time_to_hint_and_scaffold(x))

# time to finish question
sum_table['timefinishquestion'] = last['end_time']
# time to finish question
sum_table['dayfinishquestion'] = last['end_time_day']
# time to finish question
sum_table['monthfinishquestion'] = last['end_time_month']

sum_table.drop(['frishelprequest','frworkinginschool','responseisfillin',
                'frishelprequestscaffolding'],axis=1,inplace=True)
sum_table.drop(['actionid'],axis=1,inplace=True)

In [94]:
sum_table

timetaken  correct  hint  bottomhint  scaffold  \
itest_id problemid                                                    
8        104050001   54.000000        1     0           0         0   
         104050197    8.000000        1     0           0         0   
         104050201    7.000000        1     0           0         0   
         104050211   27.000000        1     0           0         0   
         104050215   10.000000        1     0           0         0   
         104050222   18.000000        0     0           0         0   
         104050223   97.000000        2     0           0         1   
         104050224  146.000001        1     0           0         5   
         104050225  378.000000        1     3           1        11   
         104050226   36.000000        1     0           0         1   
         104050227   11.000000        1     0           0         1   
         104050228   73.000000        1     0           0         0   
         104050251  265.000000        0     0           0         0   
         104050252   15.000000        1     3           1         1   
         104050253    6.000000        1     0           0         1   
         104050254    6.000000        1     0           0         2   
         104050255   37.000000        1     0           0         0   
         104050260   32.000000        1     0           0         0   
         104050273  116.000000        0     0           0         0   
         104050274   12.000000        1     0           0         1   
         104050275   14.000000        1     0           0         1   
         104050276   87.000000        1     3           1         2   
         104050277    9.000000        1     3           1         1   
         104050278  109.000000        0    10           1         0   
         104050284   56.000000        0     0           0         0   
         104050286   34.000000        1     4           1         1   
         104050287   15.000000        1     0           0         1   
         104050288    9.000000        1     0           0         2   
         104050296   45.000000        1     0           0         0   
         104050300   36.000000        1     0           0         0   
...                        ...      ...   ...         ...       ...   
7783     9322         6.000000        1     0           0         1   
         9334       572.000000        1     0           0         0   
         9335        14.000000        1     0           0         1   
         9340       260.000000        0     0           0         0   
         9341        36.000000        1     4           1         2   
         9342        15.000000        1     0           0         2   
         9345       143.000000        1     0           0         0   
         9351       437.000000        0     0           0         0   
         9352       190.000000        0     0           0         0   
         9353         8.000000        1     0           0         1   
         9354        11.000000        1     0           0         2   
         9355        13.000000        1     0           0         1   
         9357        24.000000        1     0           0         2   
         9661        21.000000        1     0           0         1   
         9662         4.000000        1     0           0         1   
         9679        27.000000        1     0           0         4   
         9680         6.000000        1     0           0         1   
         9686        30.000000        1     4           1         2   
         9687        25.000000        1     0           0         3   
         9771       456.000000        1     1           0         0   
         9772       146.000000        1     5           1         0   
         9773       303.000000        1     0           0         0   
         9774       226.000000        1     4           0         0   
         9795       397.000000        1     0

In [95]:
sum_table['actioncount'] = action_count['action_count']
matrix = sum_table.reindex(sorted(sum_table.columns), axis=1)

In [96]:
matrix.to_csv("../dataset/person_and_question_relationship.csv")